# Convert And Inference PyTorch model with CustomOps

This notebook demonstrates how to use onnxruntime-extensions to run a PyTorch model that contains operators that are not part of the ONNX standard.

## Model definition and export to ONNX
Suppose there is a model that cannot be converted because there is no matrix inverse operation in ONNX standard opset. And the model will be defined like the following.

In [1]:
import torch

class CustomInverse(torch.nn.Module):
    def forward(self, x):
        return torch.inverse(x) + x

To export this model into ONNX format, we need register a custom op handler for pytorch.onnx.exporter.

In [2]:
from torch.onnx import register_custom_op_symbolic


def my_inverse(g, self):
    return g.op("ai.onnx.contrib::Inverse", self)

register_custom_op_symbolic('::inverse', my_inverse, 1)

/depot/davisjam/data/wenxin/envs/safe-pickle/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Then, invoke the exporter

In [7]:
import io
import onnx


x0 = torch.randn(3, 3)
# Export model to ONNX
f = io.BytesIO()
t_model = CustomInverse()
torch.onnx.export(t_model, (x0, ), f, opset_version=12)
onnx_model = onnx.load(io.BytesIO(f.getvalue()))

UnsupportedOperatorError: Exporting the operator 'aten::linalg_inv' to ONNX opset version 12 is not supported. Please feel free to request support or submit a pull request on PyTorch GitHub: https://github.com/pytorch/pytorch/issues.

Now, we got a ONNX model in the memory, and it can be save into a disk file by 'onnx.save_model(onnx_model, <file_path>)

## Inference
This converted model cannot directly run the onnxruntime due to the custom operator. but it can run with onnxruntime_extensions easily.

Firstly, let define a PyOp function to intepret the custom op node in the ONNX model.

In [4]:
import numpy
from onnxruntime_extensions import onnx_op, PyOp
@onnx_op(op_type="Inverse")
def inverse(x):
    # the user custom op implementation here:
    return numpy.linalg.inv(x)


* **ONNX Inference**

In [5]:
from onnxruntime_extensions import PyOrtFunction
onnx_fn = PyOrtFunction.from_model(onnx_model)
y = onnx_fn(x0.numpy())
print(y)

[[-3.081008    0.20269153  0.42009977]
 [-3.3962293   2.5986686   2.4447646 ]
 [ 0.7805753  -0.20394287 -2.7528977 ]]


* **Compare the result with PyTorch**

In [7]:
t_y = t_model(x0)
numpy.testing.assert_almost_equal(t_y, y, decimal=5)

## Implement the customop in C++ (optional)
To make the ONNX model with the CustomOp run on all other languages supported by the ONNX Runtime and be independent of Python, a C++ implementation is needed, check [inverse.hpp](../operators/math/dlib/inverse.hpp) for an example on how to do that.

In [8]:
from onnxruntime_extensions import enable_py_op
# disable the PyOp function and run with the C++ function
enable_py_op(False)
y = onnx_fn(x0.numpy())
print(y)

[[-3.081008    0.20269153  0.42009977]
 [-3.3962293   2.5986686   2.4447646 ]
 [ 0.7805753  -0.20394287 -2.7528977 ]]
